In [2]:
# Usa la imagen oficial de Airflow
FROM apache/airflow:2.5.1

# Establece el directorio de trabajo
WORKDIR /opt/airflow
WORKDIR /opt/airflow
WORKDIR /opt/airflow

# Instala las dependencias necesarias
RUN pip install --no-cache-dir requests pandas psycopg2-binary

# Copia los archivos del DAG al directorio de DAGS de Airflow
COPY dags/ /opt/airflow/dags/

# Establece el usuario de Airflow
USER airflow


SyntaxError: invalid syntax (<ipython-input-2-9bc84c587898>, line 2)

In [10]:
!pip install apache-airflow
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime, timedelta
import requests
import pandas as pd
import psycopg2
import json
from psycopg2.extras import execute_values

# Definir la función principal que ejecutará el script
def ejecutar_script():
    # Definir URL de la API
    URL = 'https://www.datos.gov.co/resource/6y9g-d9u8.json'

    # Hacer la petición de información
    df = requests.get(URL)

    # Convertir el texto plano en JSON
    texto = df.text
    jsondata = json.loads(texto)

    # Convertir el JSON en un DataFrame de pandas
    dataframe = pd.DataFrame(jsondata)

    # Configuración de la conexión a Redshift
    conn = psycopg2.connect(
        host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
        port='5439',
        dbname='data-engineer-database',
        user='juandelbianco_coderhouse',
        password='044rmT4sNK'
    )

    cur = conn.cursor()

    # Crear la tabla si no existe
    create_table_query = '''
    CREATE TABLE IF NOT EXISTS datosapidbc (
        coddpto varchar (31),
        cod_munic varchar (31),
        a_defun varchar (31),
        sit_defun varchar (31),
        cod_inst varchar (30),
        nom_inst varchar (80),
        tipo_defun varchar (3),
        fecha_def varchar (30),
        ano varchar (43),
        mes varchar (13),
        hora varchar (31),
        minutos varchar (31),
        sexo varchar (3),
        fecha_nac varchar (30),
        est_civil varchar (3),
        edad varchar (4),
        nivel_edu varchar (2),
        ultcurfal varchar (2),
        muerteporo varchar (3),
        idpertet varchar (3),
        idpuebin varchar (3),
        codptore varchar (2),
        codmunre varchar (3),
        area_res varchar (3),
        barriofal varchar (60),
        seg_social varchar (3),
        idadmisalu varchar (3),
        idclasadmi varchar (125),
        pman_muer varchar (3),
        cons_exp varchar (3),
        peso_nac varchar (3),
        edad_madre varchar (3),
        n_hijosv varchar (3),
        n_hijosm varchar (3),
        ultcurmad varchar (3),
        man_muer varchar (3),
        comocuhec varchar (130),
        codocur varchar (2),
        codmunoc varchar (30),
        dirocuhec varchar (90),
        localocuhe varchar (30),
        c_muerte varchar (30),
        asis_med varchar (30),
        n_dir1 varchar (255),
        c_dir1 varchar (255),
        n_ant1 varchar (255),
        c_ant1 varchar (255),
        n_ant2 varchar (255),
        c_ant2 varchar (255),
        n_bas1 varchar (255),
        c_bas1 varchar (255),
        causa_666 varchar (3),
        idprofcer varchar (2),
        dd_exp varchar (2),
        mm_exp varchar (3),
        fecha_exp varchar (30),
        cau_homol varchar (2),
        gru_ed1 varchar (2),
        gru_ed2 varchar (2),
        cod_dpto varchar (30),
        emb_fal varchar (30),
        emb_sem varchar (30),
        emb_mes varchar (255),
        ocupacion varchar (255),
        c_muerteb varchar (30),
        cod_insp varchar (30),
        codigo varchar (200),
        n_ant3 varchar (255),
        c_ant3 varchar (30),
        simuertepo varchar (150),
        veredafall varchar (255),
        n_pat1 varchar (255),
        c_pat1 varchar (30),
        n_idpuebin varchar (30),
        t_dir1 varchar (30),
        m_dir1 varchar (30),
        t_ant1 varchar (30),
        m_ant1 varchar (30),
        c_ant12 varchar (30),
        c_ant22 varchar (30),
        c_dir12 varchar (30),
        hora_se varchar (30),
        codpres varchar (60),
        otrsitiode varchar (255),
        c_muerted varchar (30),
        n_pat2 varchar (30),
        c_pat2 varchar (30),
        c_muertec varchar (30),
        t_ant2 varchar (255),
        m_ant2 varchar (255),
        t_ant3 varchar (30),
        m_ant3 varchar (30),
        c_ant32 varchar (30),
        c_muertee varchar (30),
        t_pat1 varchar (30),
        m_pat1 varchar (30)
    );
    '''

    cur.execute(create_table_query)
    conn.commit()

    # Función para insertar datos
    def insertar_datos(dataframe, table_name):
        cols = ','.join(list(dataframe.columns))
        values = [tuple(x) for x in dataframe.to_numpy()]
        insert_query = f"INSERT INTO {table_name} ({cols}) VALUES %s"
        execute_values(cur, insert_query, values)
        conn.commit()

    # Inserta los datos en la tabla
    insertar_datos(dataframe, 'datosapidbc')

    # Cierra la conexión
    cur.close()
    conn.close()

# Configuración del DAG
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5)
}

dag = DAG(
    'dag_accidentes',
    default_args=default_args,
    description='DAG para cargar datos de accidentes a Redshift',
    schedule_interval=timedelta(days=1),
    start_date=datetime(2023, 1, 1),
    catchup=False
)

# Definir la tarea
tarea = PythonOperator(
    task_id='ejecutar_script',
    python_callable=ejecutar_script,
    dag=dag
)

  Using cached apache_airflow-2.10.0-py3-none-any.whl.metadata (44 kB)
  Using cached alembic-1.13.2-py3-none-any.whl.metadata (7.4 kB)
  Using cached argcomplete-3.5.0-py3-none-any.whl.metadata (16 kB)
  Using cached asgiref-3.8.1-py3-none-any.whl.metadata (9.3 kB)
  Using cached blinker-1.8.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached connexion-2.14.2-py2.py3-none-any.whl.metadata (28 kB)
  Using cached croniter-3.0.3-py2.py3-none-any.whl.metadata (28 kB)
  Using cached Flask_Caching-2.3.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached flask_session-0.5.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached flask_wtf-1.2.1-py3-none-any.whl.metadata (3.4 kB)
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached marshmallow_oneofschema-3.1.1-py3-none-any.whl.metadata (5.0 kB)
  Using cached methodtools-0.4.7-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached opentelemetry_api-1.27.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_export

ModuleNotFoundError: No module named 'airflow'

In [8]:
#Estructura de carpetas
├── Dockerfile
├── dags
│   └── dag_accidentes.py
└── requirements.txt

SyntaxError: invalid character '├' (U+251C) (<ipython-input-8-0bb68bd42c6d>, line 2)

In [9]:
#Construir y ejecutar el contenedor Docker
# Construir la imagen de Docker
!docker build -t airflow-accidentes .

# Ejecutar el contenedor de Airflow
!docker run -d -p 8080:8080 airflow-accidentes
docker-compose up -d

SyntaxError: invalid syntax (<ipython-input-9-fb386bb25582>, line 3)